In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


from teste import diretorio_teste
project_path = diretorio_teste()

In [2]:
import geopandas as gpd
import pandas as pd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Área (m²) de Áreas Verdes Públicas por Distrito

In [3]:
data_path = get_data_diretorio()
#data_path = os.path.join(diretorio_teste(),'data')

# Dependências

Este notebook é dependente dos assets resultante dos notebooks 'areas_verdes_publicas' e '../../arborizacao_viaria/carregar_dados/malha_distritos'

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

In [5]:
gdf_areas_verdes = gpd.read_parquet(
    os.path.join(
        assets_path,
        'areas_verdes_publicas',
        'areas_verdes_publicas.parquet'
    )
)

In [6]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'distrito_ibge.parquet'
    )
)

# Recortar Áreas Verdes por Distrito

In [7]:
areas_verdes_recort = gpd.overlay(
    gdf_areas_verdes,
    gdf_distritos,
    'intersection',
    keep_geom_type=True
)

In [8]:
areas_verdes_recort.sample(2)

,area_avp,id_avp,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry
13796,505.532294,13147,3550308,São Paulo,355030873,SAO MATEUS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.682299,155682,64726,"POLYGON ((346799.555 7391048.187, 346802.539 7..."
2567,2173.919692,2481,3550308,São Paulo,355030888,VILA LEOPOLDINA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,6.988733,46875,20673,"POLYGON ((323475.256 7395705.617, 323456.137 7..."


# Área dos Polígonos recortados

In [9]:
areas_verdes_recort['area_recort_avp'] = areas_verdes_recort['geometry'].area

# Calcular área por distrito

In [10]:
for distrito in gdf_distritos['CD_DIST']:
    area_avp_dist = sum(
        areas_verdes_recort['area_recort_avp']
        .loc[areas_verdes_recort['CD_DIST']==distrito]
    )

    gdf_distritos.loc[gdf_distritos['CD_DIST']==distrito,'area_avp_dist']= area_avp_dist
    

In [11]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_avp_dist
38,3550308,São Paulo,355030839,JACANA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.473932,87329,34705,"POLYGON ((339057.719 7403206.175, 339043.253 7...",108834.065711
92,3550308,São Paulo,355030893,VILA PRUDENTE,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.580243,105690,49782,"POLYGON ((339321.368 7389055.527, 339301.644 7...",403329.427273


In [12]:
gdf_distritos.loc[gdf_distritos['CD_DIST']=='355030812']

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_avp_dist
11,3550308,São Paulo,355030812,BUTANTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.887306,51715,25984,"POLYGON ((325430.01 7391267.565, 325425.704 73...",919642.274756


In [13]:
confere=areas_verdes_recort.loc[areas_verdes_recort['CD_DIST']=='355030812']
confere.sample(2)

,area_avp,id_avp,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_recort_avp
4140,477.987261,3992,3550308,São Paulo,355030812,BUTANTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.887306,51715,25984,"POLYGON ((325675.513 7392490.426, 325674.667 7...",477.987261
2863,534.464532,2765,3550308,São Paulo,355030812,BUTANTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.887306,51715,25984,"POLYGON ((324367.553 7393191.942, 324371.9 739...",534.464532


In [14]:
confere.shape

(393, 17)

In [15]:
sum(confere['area_recort_avp'])

919642.2747555246

# Salvar gdf

In [16]:
save_parquet_excel(
    gdf_distritos,
    fname='distritos_areas_verdes_publicas',
    data_path = assets_path,
    data_subpath = 'areas_verdes_publicas'
    
)